<a href="https://colab.research.google.com/github/dauten/DeepLearning/blob/master/Deep_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  from __future__ import absolute_import, division, print_function, unicode_literals

import time
import os
import random
import decimal
import sys

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import collections

# For RNN
from tensorflow.keras import layers

In [0]:
'''
Given an array of training data and an array of labels, return a model that fits
that data within (currently hardcoded) parameters

data should be a list of char array's, labels should be a numeric identifier or quality
'''
def train(data, labels, file):
  print(tf.__version__)
  N = 0 #This is the item length, we need to find number of input edges
  for each in data:
    if len(each) > N:
      N = len(each)

  M = 1 #This is the item length, we need to find number of input edges
  for each in labels:
    if each > M:
      M = each


  #the dense commands actually create neurons/nodes
  model = keras.Sequential([keras.layers.Flatten(input_shape=((N,))),
      keras.layers.Dense(2048, activation=tf.nn.relu), keras.layers.Dense(2048, activation=tf.nn.relu), keras.layers.Dense(2048, activation=tf.nn.relu), 
      keras.layers.Dense(M+1, activation=tf.nn.softmax)])

  #this tells the neural net how to update things for training
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


  #convert data and labels to ndarrays D and L
  train_images = np.ndarray((len(data), N))
  train_labels = np.ndarray((len(labels)))
  k = 0
  for i in data:
      q = 0
      for j in i:
        train_images[k][q] = ord(j)
        q+=1
      k+=1

  k = 0
  for l in labels:
      train_labels[k] = l
      k+=1

  #this is the actual 'train' step
  #you can play with epochs, increasing it takes longer but provides slightly more accuracy
  model.fit(train_images, train_labels, epochs=50)
  model.save(file+".h5")
  open(file+".pad", "w").write(str(N))

  return N

In [0]:

def trainRNN(data, labels, file):
  print(tf.__version__)
  N = 0 
  for each in data:
    if len(each) > N:
      N = len(each)

  M = 1 
  for each in labels:
    if each > M:
      M = each

  # A simple RNN model
  RNNmodel = tf.keras.Sequential()

  #First Hidden Layer
  RNNmodel.add(layers.Embedding(input_dim=N, output_dim=M+1))

    # Add a Dense layer with 10 units.
  RNNmodel.add(layers.Dense(20))
  RNNmodel.add(layers.Activation('relu'))
  RNNmodel.add(layers.BatchNormalization())

  RNNmodel.add(layers.LSTM(128))
  RNNmodel.add(layers.Activation('relu'))
  RNNmodel.add(layers.BatchNormalization())

    # Add a Dense layer with 10 units.
  RNNmodel.add(layers.Dense(20))
  RNNmodel.add(layers.Activation('softmax'))

 # Add a Dense layer with 10 units.
  #RNNmodel.add(layers.Dense(10, activation='softmax'))

  # Add a LSTM layer with 128 internal units.
  #RNNmodel.add(layers.LSTM(128, activation='softmax'))

  # Add a Dense layer with 10 units.
  #RNNmodel.add(layers.Dense(10, activation='softmax'))

  # Basic compile options
  RNNmodel.compile(loss='squared_hinge',  optimizer='sgd', metrics=['accuracy'])
 


  #convert data and labels to ndarrays D and L
  train_images = np.ndarray((len(data), N))
  train_labels = np.ndarray((len(labels)))
  k = 0
  for i in data:
      q = 0
      for j in i:
        train_images[k][q] = ord(j)
        q+=1
      k+=1

  k = 0
  for l in labels:
      train_labels[k] = l
      k+=1

  #this is the actual 'train' step
  #you can play with epochs, increasing it takes longer but provides slightly more accuracy
  model.fit(train_images, train_labels, epochs=100)

  open(file+".pad", "w").write(str(N))

  return N







In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
'''
Given a set of test data, return predictions from the (currently hardcoded) model
'''
def evaluate(test,file):

  N = 0 #This is the item length, we need to find number of input edges
  for each in test:
    if len(each) > N:
      N = len(each)

  #convert data and labels to ndarrays D and L
  train_images = np.ndarray((len(test), int(open(file+".pad", "r").read()) ))
  k = 0
  for i in test:
      q = 0

      for j in i:

        train_images[k][q] = ord(j)
        q+=1
      k+=1
  model =  keras.models.load_model(file+".h5")

  #print(model.predict(train_images))
  return model.predict(train_images)



In [0]:
def fin(filedir):
  #give a filedirectory, go through it to return tuple of data and labels
  #should consider each folder in filedir/ to be a new identifier (so len(labels)->150)
  #with the content of each file being the data
  m = 0
  data = list()
  labels = list()
  #for every folder in fildir:
  # for every item in folder:
  #   add item to data
  #   add folder to labels
  for folder in sorted(os.listdir(filedir))[:]:
    print("Beginning "+folder)
    for file in os.listdir(filedir+"/"+folder)[0:100]:
      data.append(list(open(filedir+folder+"/"+file, 'r').read())[0:20000])
      labels.append(m)
    m += 1
  return (data, labels)

In [0]:
'''
Given an array, adequetly randomize it.
Data and labels are shuffled together
'''
def shuffle(data, labels):
  N = len(data)
  for i in range(N):
    #pick 2 random numbers in [0, N) and swap those indices
    n1 = random.randrange(0, N)
    n2 = random.randrange(0, N)
    td = data[n1]
    tl = labels[n1]
    data[n1] = data[n2]
    labels[n1] = labels[n2]
    data[n2] = td
    labels[n2] = tl
  return (data, labels)

In [72]:
#once fin is implemented and the files are in place this block should
#work to train and evaluate our NN :)
data,labels = fin("/content/drive/My Drive/TEST/candidates/train/")
#data, labels = shuffle(data, labels)
#data and labels may be shuffled, just make sure they're shuffled together




Beginning bass-e-train
Beginning beck-s-train
Beginning dasovich-j-train
Beginning germany-c-train
Beginning jones-t-train
Beginning kaminski-v-train
Beginning kean-s-train
Beginning perlingiere-d-train
Beginning sanders-r-train
Beginning shackleton-s-train
Beginning shankman-j-train
Beginning symes-k-train
Beginning taylor-m-train


In [73]:
RNNpad = train(data, labels, "enron")
#print(labels)
# Just goin to leave this here for now....
#RNNpad  = trainRNN(data, labels, "RNN_enron")

1.15.0
Train on 1300 samples
Epoch 1/50
1300/1300 [==============================] - 1s 414us/sample - loss: 150.0806 - acc: 0.0900
Epoch 2/50
1300/1300 [==============================] - 0s 319us/sample - loss: 8.3316 - acc: 0.1131
Epoch 3/50
1300/1300 [==============================] - 0s 317us/sample - loss: 4.5697 - acc: 0.1346
Epoch 4/50
1300/1300 [==============================] - 0s 315us/sample - loss: 4.2818 - acc: 0.1385
Epoch 5/50
1300/1300 [==============================] - 0s 318us/sample - loss: 4.6036 - acc: 0.1585
Epoch 6/50
1300/1300 [==============================] - 0s 318us/sample - loss: 5.3167 - acc: 0.1577
Epoch 7/50
1300/1300 [==============================] - 0s 313us/sample - loss: 5.4843 - acc: 0.1377
Epoch 8/50
1300/1300 [==============================] - 0s 310us/sample - loss: 4.0784 - acc: 0.1523
Epoch 9/50
1300/1300 [==============================] - 0s 311us/sample - loss: 3.5748 - acc: 0.1577
Epoch 10/50
1300/1300 [==============================] - 0s 

In [0]:
def winner(arr):
  #given an output list of the NN return the index of the winner
  max_val = 0
  max_in = 0
  for line in range(len(arr)-1):
    #print("line is: "+str(line))
    if arr[line]*100 > max_val:
      max_val = arr[line]
      max_in = line

  
  return max_in

In [0]:
#inputs: normalized output matrix (output of evaluate), true labels, number of top guesses we should look at
#outputs: % of emails where one of our tolerance top guesses was the true label
def horseshoe(guesses, labels, tolerance, test=False):
  
  # Setup a testcase for now because the guesses aren't setup yet
  if test:
    guesses = np.random.rand(900, 9)
  
  totalCorrect = 0

  # clone the guess matrix, because we will be modifying it later
  guessClone = guesses.copy()

  for i in range(len(guesses)):
    # Grab data for the current email & the correct answer
    currentTraingItemCloned = guessClone[i]
    correctAnswer = labels[i]

    # try finding the correct answer [TOLERANCE] times
    for j in range(tolerance):
      # Find the index of the item with the maximum value
      maxIndex = np.argmax(currentTraingItemCloned)
      # if we find a correct answer, increment correct answers
      if maxIndex == correctAnswer:
        totalCorrect+= 1
        break
      else:
        # if it is not the correct answer, zero the current max, and try again
        currentTraingItemCloned[maxIndex] = 0
  
  outputAccuracy = decimal.Decimal(totalCorrect) / decimal.Decimal(len(guesses))
  # Calculate the error rate as a float between 0 and 1
  return outputAccuracy

In [0]:
'''
Given an array determine how many of each items it has--Ideally it should be a
roughly linear distribution when there's an equal amount of each label
'''
def dist(arr):
    S = sorted(arr)
    print(S)
    tracker = S[0]
    counter = 0
    for each in S:
      if tracker != each:
        print("We had "+str(counter)+" label "+str(tracker)+"'s")
        tracker = each
        counter = 1
      else:
        counter += 1

In [77]:
#print("data  :  "+str(data))
#print("labels:  "+str(labels))
data,labels = fin("/content/drive/My Drive/TEST/candidates/val/")
#known issues:
#beware overfitting (too many layers/too big of layers)
#it tends to have one or two people it like to go to a lot
#data, labels = shuffle(data, labels)

#print("data  :  "+str(data))
#print("labels:  "+str(labels))

e = evaluate(data, "enron")
print(e.round(2))

c = list()
matches = 0
pernum = [0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(len(e)):
  c.append(winner(e[i]))
for i in range(len(e)):
  if c[i] == labels[i]:
    matches += 1
    pernum[labels[i]] += 1
for i in range(labels[-1]+1):
  this = pernum[i]
  print("For items with label: "+str(i)+" we had an accuracy of "+str(this)+"%")

print(str(100*matches/len(e))+"% correct (over ten is all right)")

#dist(c)
# NOTE: with the last flag flipped to true, horseshoe generates its own test data & ignores 'e' data
h = horseshoe(e, labels, 3)
print("Horseshoe algorithm accuracy: {}%".format(h * 100))
#print("Counting top 3 we got "+str(h/len(e))+" things right")

Beginning bass-e-val
Beginning beck-s-val
Beginning dasovich-j-val
Beginning germany-c-val
Beginning jones-t-val
Beginning kaminski-v-val
Beginning kean-s-val
Beginning perlingiere-d-val
Beginning sanders-r-val
Beginning shackleton-s-val
Beginning shankman-j-val
Beginning symes-k-val
Beginning taylor-m-val
[[0.32 0.   0.   0.   0.   0.   0.   0.   0.   0.57 0.   0.11 0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.37 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.63 0.   0.  ]
 [0.84 0.   0.   0.   0.   0.   0.   0.1  0.   0.   0.06 0.   0.  ]
 [0.62 0.   0.   0.   0.   0.34 0.   0.   0.   0.01 0.   0.   0.02]
 [1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.79 0.   0.   0.01 0.   0.   0.   0.02 0.   0.   0.17 0.   0.  ]
 [0.23 0.   0.   0.04 0.   0.   0.   0.   0.   0.   0.65 0.   0.07]
 [0.77 0.   